In [4]:
# 날씨 정보를 실제 API를 이용해서 실제 날씨를 알려주도록 수정
 
import os
import json
import requests
from openai import OpenAI
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv

load_dotenv()
# 1. 설정 및 초기화
# Azure OpenAI 클라이언트 설정
client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_WEBSEARCH_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)
 
deployment_name = "gpt-4o-mini"
 
# [중요] OpenWeatherMap API 키 (https://home.openweathermap.org/api_keys)
# "YOUR_OPENWEATHER_API_KEY"를 본인의 키로 교체
OPENWEATHER_API_KEY = os.getenv("OPEN_WEATHER_API_KEY")
 
 
# 2. 실제 API 호출 함수 정의
def get_location_data(location):
    """
    OpenWeatherMap API를 통해 날씨와 타임존 오프셋 정보를 한 번에 가져오는 헬퍼 함수
    """
    if not OPENWEATHER_API_KEY:
        return None
   
    # API 요청 URL (Metric: 섭씨 기준)
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={OPENWEATHER_API_KEY}&units=metric"
   
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"API Error for {location}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Request failed: {e}")
        return None
 
def get_current_weather(location, unit="celsius"):
    """실제 API를 호출하여 날씨 정보를 반환"""
    print(f">>> [API Call] get_current_weather for: {location}")
   
    data = get_location_data(location)
   
    if data:
        # API에서 온도 추출 (기본 metric이므로 섭씨)
        temp_c = data["main"]["temp"]
        weather_desc = data["weather"][0]["description"]
       
        # 단위 변환 로직 (요청받은 단위에 따라)
        final_temp = temp_c
        if unit == "fahrenheit":
            final_temp = (temp_c * 9/5) + 32
           
        return json.dumps({
            "location": location,
            "temperature": round(final_temp, 1),
            "unit": unit,
            "description": weather_desc
        })
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
 
def get_current_time(location):
    """실제 API의 Timezone offset을 이용하여 현지 시간 계산"""
    print(f">>> [API Call] get_current_time for: {location}")
   
    data = get_location_data(location)
   
    if data:
        # OpenWeatherMap은 UTC 기준의 오프셋(초 단위)을 반환 ㅠㅜ. (예: 한국은 32400초 = 9시간)
        timezone_offset = data["timezone"]
       
        # 현재 UTC 시간에 오프셋을 더해 현지 시간 계산
        utc_now = datetime.now(timezone.utc)
        local_time = utc_now + timedelta(seconds=timezone_offset)
       
        return json.dumps({
            "location": location,
            "current_time": local_time.strftime("%Y-%m-%d %I:%M %p")
        })
    else:
        return json.dumps({"location": location, "current_time": "unknown"})
 
 
# 3. 메인 실행 로직 (병렬 함수 호출)
def run_conversation():
    # 사용자 질문
    # messages = [{"role": "user", "content": "샌프란시스코, 도쿄, 서울, 파리의 현재 날씨와 시간은?"}]
    messages = [{"role": "user", "content": "서울과 파리의 현재 날씨와 시간은?"}]
 
    # 사용할 도구(함수) 정의
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco. MUST be translated into English.",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location using timezone offset",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco. MUST be translated into English.",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]
 
    # 1차 호출: LLM이 도구 사용 여부 결정
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
 
    response_message = response.choices[0].message
    messages.append(response_message)
 
    # 도구 호출이 필요한 경우 처리
    if response_message.tool_calls:
        print(f"Model requested {len(response_message.tool_calls)} tool calls...")
       
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
           
            function_response = None
           
            if function_name == "get_current_weather":
                function_response = get_current_weather(
                    location=function_args.get("location"),
                    unit=function_args.get("unit", "celsius")
                )
            elif function_name == "get_current_time":
                function_response = get_current_time(
                    location=function_args.get("location")
                )
           
            # 결과 메시지 추가
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
           
        # 2차 호출: 도구 결과를 바탕으로 최종 답변 생성
        final_response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
        )
        return final_response.choices[0].message.content
   
    return "No tool calls needed."
 
# 실행
print(run_conversation())

Model requested 4 tool calls...
>>> [API Call] get_current_weather for: Seoul
API Error for Seoul: 401
>>> [API Call] get_current_weather for: Paris
API Error for Paris: 401
>>> [API Call] get_current_time for: Seoul
API Error for Seoul: 401
>>> [API Call] get_current_time for: Paris
API Error for Paris: 401
죄송하지만, 현재 서울과 파리의 날씨와 시간을 확인할 수 없습니다. 대신, 인터넷을 통해 기상 웹사이트나 날씨 애플리케이션에서 최신 정보를 확인해 보시는 것을 추천드립니다. 다른 질문이 있으시면 말씀해 주세요!
